# AIニュース要約 Slack Bot Notebook

このノートブックでは、OpenAI API と Slack Incoming Webhook を使って最新の AI ニュースを要約し、Slack に送信する一連の処理を実演します。

In [ ]:
# 1. 必要なライブラリをインストール
!pip install feedparser openai requests

In [ ]:
# 2. API キーの入力
from getpass import getpass
import os

OPENAI_API_KEY = getpass("OpenAI API Key: ")
SLACK_WEBHOOK_URL = getpass("Slack Webhook URL: ")

os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY
os.environ["SLACK_WEBHOOK_URL"] = SLACK_WEBHOOK_URL

In [ ]:
# 3. ライブラリのインポート & 定数定義
import feedparser
import requests
import openai
from datetime import datetime
import os

# 環境変数から読み込み
openai.api_key = os.getenv("OPENAI_API_KEY")
SLACK_WEBHOOK_URL = os.getenv("SLACK_WEBHOOK_URL")

# RSS フィード一覧 & 設定
FEEDS = {
    "MIT Technology Review AI": "https://www.technologyreview.com/tag/artificial-intelligence/feed/",
    "AI News"                 : "https://artificialintelligence-news.com/feed/",
    "ITmedia AI＋"            : "https://rss.itmedia.co.jp/rss/2.0/aiplus.xml"
}
MAX_ARTICLES = 3

In [ ]:
# 4. 関数定義
def fetch_latest_entries(feed_url, max_items=3):
    feed = feedparser.parse(feed_url)
    entries = []
    for entry in feed.entries[:max_items]:
        title = entry.get("title", "").strip()
        summary = entry.get("summary", "").strip()
        link = entry.get("link", "").strip()
        entries.append(f"- <{link}|{title}>: {summary}")
    return entries

def summarize_with_chatgpt(site_name, items):
    prompt = (
        f"以下は「{site_name}」の最新AIニュース記事の見出しと概要です。\n"
        "これらを技術トレンドや注目ポイントがひと目でわかるよう、"
        "日本語で250文字以内の箇条書き要約にしてください。\n\n"
        + "\n".join(items)
    )
    response = openai.chat.completions.create(
        model="gpt-4o-mini",
        temperature=0.3,
        messages=[
            {"role": "system", "content": "あなたは有能な技術ニュース要約アシスタントです。"},
            {"role": "user", "content": prompt}
        ]
    )
    return response.choices[0].message.content.strip()

def post_to_slack(text):
    payload = {"text": text}
    resp = requests.post(SLACK_WEBHOOK_URL, json=payload)
    resp.raise_for_status()

In [ ]:
# 5. メイン処理の実行
now = datetime.now().strftime("%Y-%m-%d %H:%M")
blocks = [f"*AIニュース要約* (`{now}`)"]

for site, url in FEEDS.items():
    entries = fetch_latest_entries(url, MAX_ARTICLES)
    if not entries:
        blocks.append(f"> {site} のニュースを取得できませんでした。")
        continue
    summary = summarize_with_chatgpt(site, entries)
    blocks.append(f"*{site}* の最新要約:\n{summary}")
    blocks.append("")

message = "\n".join(blocks)
post_to_slack(message)
print("Slackへ送信しました。")

In [ ]:
# 6. 要約内容をノートブック上でプレビュー（任意）
from IPython.display import Markdown, display

display(Markdown("\n".join(blocks)))